In [3]:
!pip install gcloud auth application-default login

  Using cached gcloud-0.18.3.tar.gz (454 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached auth-0.5.3-py3-none-any.whl.metadata (7.6 kB)


ERROR: Could not find a version that satisfies the requirement application-default (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for application-default


In [4]:
from faker import Faker
import random
import string
import pandas as pd
from google.cloud import storage

In [5]:
fake = Faker()

# ---------- Step 1: Generate Random Password ----------
def generate_password(length=10):
    chars = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(chars) for _ in range(length))

In [6]:
# ---------- Step 2: Generate Employee Data ----------
def generate_employee_data(n=100):
    departments = ["HR", "Finance", "Engineering", "Sales", "Marketing", "IT", "Admin"]
    employees = []

    for i in range(1, n+1):
        emp = {
            "employee_id": i,
            "name": fake.name(),
            "department": random.choice(departments),
            "salary": random.randint(30000, 120000),
            "email": fake.email(),
            "joining_date": fake.date_between(start_date="-5y", end_date="today"),
            "password": generate_password(10)
        }
        employees.append(emp)

    return pd.DataFrame(employees)

In [ ]:
# ---------- Step 3: Save to CSV ----------
df = generate_employee_data(50)  # Generate 50 employees
csv_file = "employee_data.csv"
# df.to_csv(csv_file, index=False)  # this will save the data into your local directory

In [24]:
df.head()

,employee_id,name,department,salary,email,joining_date,password
0,1,John Harris,Admin,114703,randolphsara@example.org,2025-02-13,"?Tj+N|p""d("
1,2,Casey Campbell,Engineering,62588,gibsonjose@example.net,2025-08-09,54nu~l}^6z
2,3,Ann Garcia,Finance,91765,autumnmartinez@example.org,2022-09-29,cUQoD|s$>9
3,4,Brian Taylor,Marketing,62393,piercerichard@example.com,2024-08-30,BgAb4q!7e1
4,5,Debbie Warren,Engineering,86150,wweaver@example.net,2024-03-09,"~#2*V""L{Du"


In [9]:
# ---------- Step 4: Upload to GCS ----------
def upload_to_gcs(bucket_name, source_file, destination_blob):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob)
    blob.upload_from_filename(source_file)
    print(f"✅ File {source_file} uploaded to gs://{bucket_name}/{destination_blob}")

In [20]:
# Example usage
bucket_name = "data_gcp_bucket"  # 🔹 Replace with your bucket name
destination_blob = "employee_data.csv"
upload_to_gcs(bucket_name, csv_file, destination_blob)

✅ File employee_data.csv uploaded to gs://data_gcp_bucket/employee_data.csv


Upload CSV to GCS in the new bucket folder by creating if not exists

In [ ]:
from faker import Faker
import random
import string
import pandas as pd
from google.cloud import storage

fake = Faker()

# ---------- Step 1: Generate Random Password ----------
def generate_password(length=10):
    chars = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(chars) for _ in range(length))

# ---------- Step 2: Generate Employee Data ----------
def generate_employee_data(n=100):
    departments = ["HR", "Finance", "Engineering", "Sales", "Marketing", "IT", "Admin"]
    employees = []

    for i in range(1, n+1):
        emp = {
            "employee_id": i,
            "name": fake.name(),
            "department": random.choice(departments),
            "salary": random.randint(30000, 120000),
            "email": fake.email(),
            "joining_date": fake.date_between(start_date="-5y", end_date="today"),
            "password": generate_password(10)
        }
        employees.append(emp)

    return pd.DataFrame(employees)

# ---------- Step 3: Save to CSV ----------
df = generate_employee_data(50)
csv_file = "employee_data.csv"
# df.to_csv(csv_file, index=False)

# ---------- Step 4: Upload to GCS (subfolder) ----------
def upload_to_gcs(bucket_name, sub_folder, source_file, destination_file):
    client = storage.Client()

    # Ensure bucket exists
    try:
        bucket = client.get_bucket(bucket_name)
        print(f"✅ Bucket {bucket_name} exists")
    except Exception:
        bucket = client.create_bucket(bucket_name, location="US")
        print(f"🆕 Bucket {bucket_name} created")

    # Upload file inside sub-folder
    destination_blob = f"{sub_folder}/{destination_file}"
    blob = bucket.blob(destination_blob)
    blob.upload_from_filename(source_file)
    print(f"✅ File {source_file} uploaded to gs://{bucket_name}/{destination_blob}")

# Example usage
bucket_name = "data_gcp_bucket"           # main bucket
sub_folder = "2025/07/pathmatics"              # subfolder path
upload_to_gcs(bucket_name, sub_folder, csv_file, "employee_data.csv")


✅ Bucket data_gcp_bucket exists
✅ File employee_data.csv uploaded to gs://data_gcp_bucket/2025/07/pathmatics/employee_data.csv
